![tower bridge](london.jpg)

London, or as the Romans called it "Londonium"! Home to [over 8.5 million residents](https://www.ons.gov.uk/peoplepopulationandcommunity/populationandmigration/populationestimates/bulletins/populationandhouseholdestimatesenglandandwales/census2021unroundeddata#population-and-household-estimates-england-and-wales-data) who speak over [300 languages](https://web.archive.org/web/20080924084621/http://www.cilt.org.uk/faqs/langspoken.htm). While the City of London is a little over one square mile (hence its nickname "The Square Mile"), Greater London has grown to encompass 32 boroughs spanning a total area of 606 square miles! 

![underground train leaving a platform](tube.jpg)

Given the city's roads were originally designed for horse and cart, this area and population growth has required the development of an efficient public transport system! Since the year 2000, this has been through the local government body called **Transport for London**, or *TfL*, which is managed by the London Mayor's office. Their remit covers the London Underground, Overground, Docklands Light Railway (DLR), buses, trams, river services (clipper and [Emirates Airline cable car](https://en.wikipedia.org/wiki/London_cable_car)), roads, and even taxis.

The Mayor of London's office make their data available to the public [here](https://data.london.gov.uk/dataset). In this project, you will work with a slightly modified version of a dataset containing information about public transport journey volume by transport type. 

The data has been loaded into an **AWS Redshift** database called `tfl` with a single table called `journeys`, including the following data:

## tfl.journeys

| Column | Definition | Data type |
|--------|------------|-----------|
| `month`| Month in number format, e.g., `1` equals January | `INTEGER` |
| `year` | Year | `INTEGER` |
| `days` | Number of days in the given month | `INTEGER` |
| `report_date` | Date that the data was reported | `DATE` |
| `journey_type` | Method of transport used | `VARCHAR` |
| `journeys_millions` | Millions of journeys, measured in decimals | `FLOAT` |

You will execute SQL queries to answer three questions, as listed in the instructions.

In [11]:
-- Visualizing data
SELECT *
FROM tfl.journeys
LIMIT 10

,month,year,days,report_date,journey_type,journeys_millions
0,1,2010,31,2010-06-30 00:00:00+00:00,Underground & DLR,96.836391
1,2,2010,28,2010-06-30 00:00:00+00:00,Underground & DLR,90.330504
2,3,2010,31,2010-06-30 00:00:00+00:00,Underground & DLR,90.038014
3,4,2010,30,2010-09-30 00:00:00+00:00,Underground & DLR,92.544093
4,5,2010,31,2010-09-30 00:00:00+00:00,Underground & DLR,88.662911
5,6,2010,30,2010-09-30 00:00:00+00:00,Underground & DLR,86.375702
6,7,2010,31,2010-12-31 00:00:00+00:00,Underground & DLR,94.986035
7,8,2010,31,2010-12-31 00:00:00+00:00,Underground & DLR,97.024794
8,9,2010,30,2010-12-31 00:00:00+00:00,Underground & DLR,97.029640
9,10,2010,31,2011-03-31 00:00:00+00:00,Underground & DLR,77.369979


In [12]:
-- most_popular_transport_types
SELECT journey_type,
		ROUND(SUM(journeys_millions),2) total_journeys_millions -- Use "ROUND" to be more legible
FROM tfl.journeys
GROUP BY journey_type
ORDER BY SUM(journeys_millions) DESC

,journey_type,total_journeys_millions
0,Bus,24905.19
1,Underground & DLR,15020.47
2,Overground,1666.85
3,TfL Rail,411.31
4,Tram,314.69
5,Emirates Airline,14.58


In [13]:
-- emirates_airline_popularity
SELECT month,
		year,
		ROUND(SUM(journeys_millions),2) rounded_journeys_millions
FROM tfl.journeys
WHERE journey_type = 'Emirates Airline' AND journeys_millions IS NOT NULL
GROUP BY month, year
ORDER BY SUM(journeys_millions) DESC
LIMIT 5

,month,year,rounded_journeys_millions
0,5,2012,0.53
1,6,2012,0.38
2,4,2012,0.24
3,5,2013,0.19
4,5,2015,0.19


In [14]:
-- least_popular_years_tube
SELECT year,
		journey_type,
		ROUND(SUM(journeys_millions),2) total_journeys_millions
FROM tfl.journeys
WHERE journey_type = 'Underground & DLR' AND journeys_millions IS NOT NULL
GROUP BY year, journey_type
ORDER BY SUM(journeys_millions) ASC
LIMIT 5

,year,journey_type,total_journeys_millions
0,2020,Underground & DLR,310.18
1,2021,Underground & DLR,748.45
2,2022,Underground & DLR,1064.86
3,2010,Underground & DLR,1096.15
4,2011,Underground & DLR,1156.65
